In [2]:
import urllib.request
import pandas as pd
from pandas import json_normalize
import json
import time

#获取themes对应类别
themes_str = {"taxes" : ["ECON_TAXATION",],
"unemployment" : [ "UNEMPLOYMENT", ],
"domesticeconomy" : ["ECON_BANKRUPTCY", "ECON_BOYCOTT", "ECON_COST_OF_LIVING", "ECON_CUTOUTLOOK", "ECON_DEREGULATION", "ECON_EARNINGSREPORT", "ECON_ENTREPRENEURSHIP", "ECON_HOUSING_PRICES", "ECON_INFORMAL_ECONOMY", "ECON_IPO", "ECON_INTEREST_RATE", "ECON_MONOPOLY", "ECON_MOU", "ECON_NATIONALIZE", "ECON_PRICECONTROL", "ECON_REMITTANCE", "ECON_STOCKMARKET", "ECON_SUBSIDIES", "ECON_UNIONS", "SLFID_ECONOMIC_DEVELOPMENT", "SLFID_ECONOMIC_POWER", "SOC_ECONCOOP"],
"trade" : ["ECON_TRADE_DISPUTE", "ECON_FOREIGNINVEST", "ECON_FREETRADE", "ECON_CURRENCY_EXCHANGE_RATE", "ECON_CURRENCY_RESERVES", "ECON_DEBT"],
"terrorism" : ["TAX_TERROR_GROUP", "SUICIDE_ATTACK", "EXTREMISM", "JIHAD", "TERROR", "WMD"],
"military" : ["ACT_FORCEPOSTURE", "ARMEDCONFLICT", "BLOCKADE", "CEASEFIRE", "MILITARY", "MILITARY_COOPERATION", "PEACEKEEPING", "RELEASE_HOSTAGE", "SEIGE", "SLFID_MILITARY_BUILDUP", "SLFID_MILITARY_READINESS", "SLFID_MILITARY_SPENDING", "SLFID_PEACE_BUILDING", "TAX_MILITARY_TITLE"],
"internationalrelations" : ["GOV_INTERGOVERNMENTAL", "SOC_DIPLOMCOOP", "RELATIONS"],
"immigration/refugees" : ["BORDER", "CHECKPOINT", "DISPLACED",  "EXILE", "IMMIGRATION", "REFUGEES", "SOC_FORCEDRELOCATION", "SOC_MASSMIGRATION", "UNREST_CHECKPOINT", "UNREST_CLOSINGBORDER"],
"healthcare" : ["GENERAL_HEALTH", "HEALTH_SEXTRANSDISEASE", "HEALTH_VACCINATION", "MEDICAL", "MEDICAL_SECURITY"],
"guncontrol" : ["FIREARM_OWNERSHIP", "MIL_SELF_IDENTIFIED_ARMS_DEAL", "MIL_WEAPONS_PROLIFERATION"],
"drug" : ["CRIME_ILLEGAL_DRUGS", "DRUG_TRADE", "TAX_CARTELS", "CRIME_CARTELS"],
"policesystem" : ["UNREST_POLICEBRUTALITY", "SECURITY_SERVICES"],
"racism" : ["DISCRIMINATION", "HATE_SPEECH"],
"civilliberties" : ["GENDER_VIOLENCE", "LGBT", "MOVEMENT_SOCIAL",  "MOVEMENT_WOMENS", "SLFID_CIVIL_LIBERTIES"],
"environment" :  ["ENV_BIOFUEL", "ENV_CARBONCAPTURE", "ENV_CLIMATECHANGE",  "ENV_COAL", "ENV_DEFORESTATION", "ENV_FISHERY", "ENV_FORESTRY", "ENV_GEOTHERMAL", "ENV_GREEN", "ENV_HYDRO", "ENV_METALS", "ENV_MINING", "ENV_NATURALGAS", "ENV_NUCLEARPOWER", "ENV_OIL", "ENV_OVERFISH", "ENV_POACHING", "ENV_WATERWAYS ", "ENV_SOLAR", "ENV_SPECIESENDANGERED", "ENV_SPECIESEXTINCT", "ENV_WINDPOWER", "FUELPRICES", "MOVEMENT_ENVIRONMENTAL", "SELF_IDENTIFIED_ENVIRON_DISASTER", "SLFID_MINERAL_RESOURCES", "SLFID_NATURAL_RESOURCES", "WATER_SECURITY"],
"partypolitics" : ["TAX_POLITICAL_PARTY"],
"electionfraud" : ["ELECTION_FRAUD"],
"education" : ["EDUCATION"],
"media/internet" : ["CYBER_ATTACK",  "INTERNET_BLACKOUT", "INTERNET_CENSORSHIP", "MEDIA_CENSORSHIP", "MEDIA_MSM", "MEDIA_SOCIAL", "SURVEILLANCE", "FREESPEECH"],
}
themes_df = json_normalize(themes_str)

#创建空dataframe
import pandas as pd
themes_pd = pd.DataFrame(columns=['new_cat','themes'])
#插入dataframe
for i in themes_str:
    for j in themes_str[i]:
        themes_pd.loc[len(themes_pd)] = [i,j]
        
themes_pd.to_csv('new_cat.csv',index = False)

In [74]:
for i in themes_pd['new_cat'].unique():
    print(i)

taxes
unemployment
domesticeconomy
trade
terrorism
military
internationalrelations
immigration/refugees
healthcare
guncontrol
drug
policesystem
racism
civilliberties
environment
partypolitics
electionfraud
education
media/internet


In [75]:
themes_pd

,new_cat,themes
0,taxes,ECON_TAXATION
1,unemployment,UNEMPLOYMENT
2,domesticeconomy,ECON_BANKRUPTCY
3,domesticeconomy,ECON_BOYCOTT
4,domesticeconomy,ECON_COST_OF_LIVING
...,...,...
118,media/internet,MEDIA_CENSORSHIP
119,media/internet,MEDIA_MSM
120,media/internet,MEDIA_SOCIAL
121,media/internet,SURVEILLANCE


In [ ]:
#查询数据库，关联两张表获取各类别theme的个数
#仅查询选取的19个城市
select 
day_date,
country_name,
report_name,
new_cat,
sum(nums) as sumnum
from
`dotted-marking-340807.test_20210101_202103031.new_cat`  A
join 
(   --查询被报道国包含这些国家的记录
    select
    day_date,
    A.en_name as country_name,
    B.en_name as report_name,
    themes_solo,
    nums
    from
    (   --查询报道过包含这些国家的的记录
        select
        day_date,
        en_name,
        report_code,
        themes_solo,
        nums
        from
        `dotted-marking-340807.test_20210101_202103031.wangyu_2017_themes`  A
        join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.country_code = B.country_code
    )A join `dotted-marking-340807.test_20210101_202103031.APEC_country` B on A.report_code = B.country_code
)B
 on
B.themes_solo like concat(A.themes,'%') --查询themes包含这些前缀的类别
group by
day_date,
country_name,
report_name,
new_cat

In [3]:
import numpy as np
#读取2017的结果
#加载文件名，结果名和结果文件名

file_name = '2017_cat_nums.csv'
result_file_name = '2017_cat_new_nums.csv'

#获取数据
alldata = pd.read_csv(file_name)  
APEC_country = pd.read_csv('APEC_country.csv')
APEC_country.iloc[1,4] = 'China, Hong Kong SAR'
APEC_country = APEC_country[['country_code','en_name']]

#去掉文莱
country_list = APEC_country['en_name'].unique()
country_list.sort()
country_list = np.delete(country_list, 1)
#获取日期和分类
day = alldata['day_date'].unique()
#20170829数据缺失 手动加入
day = np.append(day,20170829)
day.sort()
cat = alldata['new_cat'].unique()
cat.sort()

#抬头
result_pd = pd.DataFrame(columns = ['country_name','report_name','new_cat','num_list'])

In [4]:
APEC_country

,country_code,en_name
0,CH,China
1,HK,Hong Kong
2,TW,Taiwan
3,AS,Australia
4,BX,Brunei
5,CA,Canada
6,CI,Chile
7,ID,Indonesia
8,JA,Japan
9,KS,South Korea


In [52]:
#插入数据
result_pd = pd.DataFrame(columns = ['country_name','report_name','new_cat','num_list'])
for country_A in country_list:
    for country_B in country_list:#前两层判断国家
        for now_cat in cat:#判断类别，并生成时间列表
            now_list = []
            now_list.append(country_A)
            now_list.append(country_B)
            now_list.append(now_cat)
            print(country_A,country_B,now_cat)
            timelist = []
            for time in day:
                #print(time)
                if(time == 20170829):#缺失时间补-1
                    timelist.append(-1)
                    continue
                if(result[(result['day_date'] == time) & (result['new_cat'] == now_cat) & (result['country_name'] == country_A) & (result['report_name'] == country_B)].empty == 0):
                    timelist.append(result[(result['day_date'] == time) & (result['new_cat'] == now_cat) & (result['country_name'] == country_A) & (result['report_name'] == country_B)].iloc[0,4])
                else:
                    timelist.append(0)
            now_list.append(timelist)
            result_pd.loc[len(result_pd)] = now_list
            print(now_list)
            
result_pd.to_csv( result_file_name , index = False)

Australia Australia civilliberties
['Australia', 'Australia', 'civilliberties', [108, 76, 207, 330, 109, 287, 52, 436, 354, 199, 282, 207, 197, 136, 64, 87, 183, 355, 163, 147, 98, 282, 184, 137, 551, 254, 142, 226, 627, 187, 162, 158, 201, 527, 411, 325, 513, 732, 293, 255, 709, 237, 301, 209, 247, 495, 480, 265, 397, 104, 472, 272, 386, 367, 447, 571, 491, 1094, 617, 325, 654, 375, 510, 315, 2164, 356, 649, 276, 418, 292, 149, 354, 397, 996, 285, 644, 361, 352, 379, 1310, 710, 284, 184, 65, 300, 328, 806, 1086, 495, 234, 233, 60, 783, 382, 607, 190, 269, 76, 208, 479, 465, 105, 355, 75, 68, 147, 210, 154, 412, 773, 607, 185, 199, 205, 141, 163, 261, 250, 111, 71, 202, 198, 257, 401, 99, 36, 313, 235, 171, 187, 888, 291, 390, 146, 434, 192, 182, 232, 554, 265, 158, 326, 360, 534, 273, 230, 201, 296, 192, 219, 271, 143, 313, 198, 176, 235, 184, 267, 340, 385, 75, 151, 392, 339, 385, 235, 308, 73, 107, 221, 115, 116, 143, 119, 67, 195, 391, 272, 154, 98, 285, 211, 217, 256, 149, 125, 10

['Australia', 'Australia', 'environment', [129, 491, 506, 995, 1811, 875, 296, 567, 1029, 1196, 780, 1024, 759, 835, 797, 1369, 1536, 1210, 1100, 716, 344, 252, 1226, 1202, 1385, 960, 1083, 874, 534, 775, 1457, 1484, 1585, 1957, 894, 1297, 1906, 1823, 1479, 2450, 1852, 687, 817, 3385, 2075, 1671, 2142, 2471, 703, 1041, 1009, 1751, 2629, 1852, 2315, 1144, 2366, 2454, 2097, 2361, 2513, 1101, 1003, 592, 1275, 1540, 1235, 2652, 1891, 705, 1177, 1556, 3623, 1643, 2052, 1992, 1308, 1463, 1993, 1012, 1907, 1234, 1107, 765, 562, 1716, 1253, 1854, 2567, 1655, 803, 820, 2134, 1603, 1067, 1037, 792, 452, 1566, 1408, 1054, 2216, 1307, 406, 452, 483, 1117, 1581, 1331, 1567, 1065, 711, 513, 1123, 811, 1649, 2003, 1231, 301, 357, 1020, 1506, 925, 1170, 1142, 262, 505, 946, 1237, 930, 1400, 905, 317, 528, 1330, 1913, 1331, 1456, 1179, 768, 993, 2059, 1292, 924, 826, 2162, 1195, 628, 1833, 2361, 1343, 1703, 2240, 806, 471, 2394, 1759, 2065, 2240, 2861, 309, 1153, 1588, 1870, 2801, 1321, 1185, 634, 1052

['Australia', 'Australia', 'media/internet', [1161, 2260, 2060, 2934, 2451, 2397, 916, 1594, 3206, 2740, 1822, 2630, 2573, 1706, 1816, 3009, 3823, 3287, 3807, 3250, 1484, 2002, 3110, 2693, 2893, 2297, 2252, 1506, 1092, 2918, 2904, 3000, 4098, 3460, 2630, 2597, 2822, 3214, 3201, 2556, 3720, 2000, 2321, 4651, 3753, 3342, 3893, 4342, 1644, 1368, 3005, 3876, 2693, 2482, 2987, 2271, 2563, 3199, 3106, 3426, 4642, 2980, 1293, 1191, 4087, 2820, 3135, 3310, 2606, 2841, 2431, 3108, 4658, 3506, 4320, 3621, 2066, 2056, 4133, 4561, 3092, 2824, 3244, 1024, 2365, 4415, 5007, 3740, 3833, 3201, 1565, 1765, 3680, 4013, 2346, 2924, 2219, 2119, 1940, 4770, 3007, 3479, 2460, 2018, 1042, 1412, 2239, 2888, 3478, 2348, 3111, 1799, 2216, 4101, 1960, 3396, 4157, 1886, 729, 988, 1983, 2414, 2017, 3036, 1723, 1405, 2556, 3910, 3595, 3851, 4489, 3682, 1837, 3001, 2608, 2739, 2183, 3204, 3474, 1501, 1013, 2856, 2035, 3975, 2323, 2760, 1757, 1830, 3843, 3647, 3469, 2885, 2426, 1330, 1601, 3315, 3491, 2732, 3259, 201

['Australia', 'Australia', 'racism', [486, 69, 61, 213, 150, 80, 221, 295, 56, 200, 165, 149, 442, 46, 823, 680, 153, 307, 349, 315, 71, 567, 514, 329, 556, 402, 540, 226, 144, 389, 2015, 1266, 127, 965, 90, 270, 411, 694, 498, 288, 1490, 1135, 366, 762, 1143, 670, 1369, 434, 268, 120, 182, 366, 604, 184, 351, 451, 958, 428, 1200, 270, 735, 260, 279, 572, 1375, 428, 572, 540, 385, 724, 557, 1582, 808, 221, 258, 687, 150, 498, 636, 1355, 779, 201, 709, 197, 83, 1159, 326, 381, 883, 419, 291, 351, 1239, 818, 323, 219, 121, 36, 713, 190, 121, 126, 333, 160, 170, 188, 332, 211, 871, 528, 513, 134, 878, 156, 499, 229, 276, 137, 69, 131, 356, 93, 80, 171, 126, 70, 143, 812, 148, 339, 474, 163, 231, 80, 679, 257, 369, 323, 968, 215, 82, 220, 233, 286, 455, 845, 566, 182, 295, 168, 317, 272, 1127, 304, 530, 503, 135, 189, 287, 222, 216, 86, 168, 112, 255, 232, 114, 88, 333, 464, 121, 194, 210, 94, 42, 60, 167, 79, 58, 94, 103, 242, 47, 221, 99, 116, 96, 105, 229, 120, 304, 170, 290, 64, 79, 33

['Australia', 'Canada', 'civilliberties', [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 3, 0, 3, 0, 0, 0, 1, 2, 1, 0, 1, 4, 2, 5, 3, 0, 0, 0, 4, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 5, 0, 0, 0, 0, 0, 8, 2, 0, 4, 0, 7, 1, 0, 133, 1, 0, 0, 0, 0, 1, 5, 2, 4, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 4, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0, 1, 0, 1, 0, 0, 2, 0, 11, 9, 0, 0, 0, 0, 1, 3, 125, 5, 0, 1, 2, 0, 1, 0, 6, 4, 1, 0, 0, 0, 1, 0, 4, 3, 1, 6, 1, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 1, 0, 0, 0, 3, 1, 0, 1, 2, 1, 0, 1, 9, 2, 1, 0, 2, 0, 0, 0, 3, 1, 1, 0, 3, 0, 0, 0, 0, 1, 0, 0, 1, 2, 1, 3, 1, 0, 0, 0, 5, 0, 1, 0, 1, 0, 0, 14, 1, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 11, 0, 0, 1, 1, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 2, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 4, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 1, 4, 0, 0, 0, 1, 1, 2, 3, 2, 0, 0, 1, 0, 0, 0, 1, 0, 2, 0, 1, 2, 0, 0, 1, 0, 2, 0, 1, 0, 0, 31, 19, 2, 0, 1, 0, 9, 0, 1,

...

['Australia', 'Chile', 'internationalrelations', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0

['Australia', 'Chile', 'terrorism', [0, 2, 0, 0, 2, 0, 0, 12, 0, 0, 0, 3, 7, 4, 0, 0, 0, 3, 0, 0, 0, 0, 17, 11, 110, 2, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 2, 7, 1, 0, 4, 2, 0, 8, 0, 4, 1, 1, 0, 1, 0, 1, 1, 1, 3, 1, 1, 0, 0, 0, 2, 2, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 2, 0, 0, 1, 0, 1, 1, 0, 0, 0, 3, 1, 0, 0, 1, 0, 0, 1, 2, 1, 0, 8, 0, 0, 4, 1, 0, 0, 0, 0, 5, 0, 0, 0, 0, 7, 0, 1, 4, 1, 1, 2, 0, 0, 0, 0, 1, 0, 0, 4, 2, 0, 1, 2, 0, 0, 0, 0, 0, 2, 0, 2, 6, 0, 2, 5, 12, 0, 0, 0, 0, 0, 6, 0, 2, 5, 0, 2, 0, 0, 0, 3, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 4, 0, 0, 0, 0, 3, 0, 0, 4, 0, 0, 0, 0, 1, 6, 2, 0, 3, 1, 0, 1, 0, 0, 0, 0, 0, 10, 0, 0, 7, 3, 0, 3, 9, 0, 0, 1, 0, 0, 0, 6, 18, 0, 0, 1, 0, 0, 5, 0, 0, 0, 0, 7, -1, 0, 0, 2, 5, 0, 1, 0, 1, 1, 1, 5, 3, 0, 0, 6, 4, 0, 2, 0, 0, 0, 0, 0, 0, 2, 4, 1, 3, 1, 0, 0, 0, 0, 0, 0, 0, 10, 6, 0, 8, 0, 4, 3, 0, 0, 0, 0, 2, 5, 0, 0, 1, 22, 0, 2, 4, 5, 0, 1, 0, 1, 0, 3, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 1, 0

['Australia', 'China', 'electionfraud', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

['Australia', 'China', 'military', [36, 56, 89, 20, 15, 11, 86, 40, 35, 21, 98, 98, 28, 83, 4, 20, 32, 100, 41, 16, 260, 36, 45, 85, 18, 16, 24, 1, 7, 9, 25, 30, 18, 16, 3, 30, 47, 60, 73, 175, 11, 2, 17, 27, 31, 32, 35, 32, 33, 6, 17, 44, 11, 12, 7, 16, 18, 61, 18, 71, 13, 79, 64, 16, 33, 73, 46, 34, 16, 272, 7, 58, 14, 216, 17, 1, 1, 13, 23, 62, 167, 66, 37, 16, 131, 28, 120, 223, 270, 183, 32, 12, 17, 8, 162, 23, 29, 27, 10, 26, 71, 70, 209, 18, 4, 4, 32, 122, 19, 64, 40, 18, 13, 29, 18, 20, 18, 9, 1, 14, 16, 10, 24, 17, 14, 14, 23, 22, 31, 4, 48, 20, 2, 6, 23, 34, 46, 29, 56, 29, 15, 24, 45, 208, 67, 20, 23, 61, 54, 75, 41, 64, 56, 422, 10, 234, 62, 83, 50, 15, 6, 61, 25, 317, 29, 23, 76, 64, 24, 7, 30, 19, 13, 31, 53, 15, 26, 60, 45, 30, 54, 34, 13, 55, 46, 40, 22, 43, 17, 36, 22, 18, 96, 57, 33, 35, 18, 33, 29, 23, 15, 17, 16, 6, 17, 111, 24, 16, 17, 2, 82, 62, 38, 67, 30, 28, 6, 8, 199, 65, 75, 67, 7, 4, 6, 39, 122, 32, 30, 24, 8, 18, 33, 82, 11, 15, 45, 14, 7, 21, -1, 7, 117, 1

KeyboardInterrupt: 

In [60]:
cat

array(['civilliberties', 'domesticeconomy', 'drug', 'education',
       'electionfraud', 'environment', 'guncontrol', 'healthcare',
       'immigration/refugees', 'internationalrelations', 'media/internet',
       'military', 'partypolitics', 'policesystem', 'racism', 'taxes',
       'terrorism', 'trade', 'unemployment'], dtype=object)

In [66]:
alldata

,country_name,report_name,new_cat,num_list
0,Australia,Australia,civilliberties,"[108, 76, 207, 330, 109, 287, 52, 436, 354, 19..."
1,Australia,Australia,domesticeconomy,"[334, 473, 637, 1012, 713, 575, 216, 292, 838,..."
2,Australia,Canada,civilliberties,"[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0, 0, ..."
3,Australia,Canada,domesticeconomy,"[0, 0, 13, 3, 5, 5, 2, 2, 3, 4, 1, 7, 1, 1, 1,..."
4,Australia,Chile,civilliberties,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, ..."
...,...,...,...,...
319,Vietnam,Singapore,unemployment,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
320,Vietnam,South Korea,unemployment,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
321,Vietnam,Thailand,unemployment,"[2, 1, 0, 4, 0, 0, 3, 2, 1, 1, 4, 0, 4, 2, 3, ..."
322,Vietnam,United States,unemployment,"[4, 0, 2, 0, 0, 1, 1, 0, 0, 3, 1, 2, 1, 0, 1, ..."


In [65]:
#将所有日期数据合并
alldata = pd.DataFrame(columns=['country_name','report_name','new_cat','num_list'])
for i in range(1,11):
    result = pd.read_csv(str(i) + '_2017_cat_new_nums.csv')
    alldata=pd.concat([alldata, result])

In [67]:
alldata.to_csv('2017_cat_new_nums.csv', index = False)